In [10]:
import os
import torch
import trimesh
import numpy as np

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_mesh
from shap_e.util.image_util import load_image
from shap_e.rendering.ply_util import write_ply

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [01:19<00:00, 11.7MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [4]:
output_folder = "plys"  # 원하는 폴더 이름으로 변경
os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성

In [7]:
batch_size = 1
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("example_data/corgi.png")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [11]:
mesh = decode_latent_mesh(xm, latents).tri_mesh()

# Get the vertex colors
red_channel = mesh.vertex_channels['R']
green_channel = mesh.vertex_channels['G']
blue_channel = mesh.vertex_channels['B']

mesh = trimesh.Trimesh(vertices=mesh.verts, faces=mesh.faces)

# Add the vertex colors to the PLY file
mesh.visual.vertex_colors = np.column_stack((red_channel, green_channel, blue_channel))

file_name = f'scale_{guidance_scale}.ply'
file_path = os.path.join(output_folder, file_name)
mesh.export(file_path)

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 23826\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nproperty uchar alpha\nelement face 47648\nproperty list uchar int vertex_indices\nend_header\nk\xe4\x9a\xbe\x93\xb6$\xbf\xe8\x0b\xfa>R\x1b\x00\xff3\x1d\x93\xbe\xa2\xae(\xbf\xe8\x0b\xfa>&\x17\x02\xff\xcb\xae\x93\xbe\xa2\xae(\xbf\x04\xfe\x00?+\x19\x03\xffv\x07\x94\xbe\xa2\xae(\xbf\x14\xf6\x04?6\x1d\x03\xffP[\x94\xbe\xa2\xae(\xbf$\xee\x08?G!\x03\xff\xc4\xb5\x94\xbe\xa2\xae(\xbf4\xe6\x0c?X%\x03\xff\x9c\xa9\x95\xbe\xa2\xae(\xbfD\xde\x10?f)\x02\xffLo\x96\xbe\xa2\xae(\xbfT\xd6\x14?r+\x01\xff\xe03\x96\xbe\xa2\xae(\xbfd\xce\x18?v,\x01\xff(\xf2\x94\xbe\xa2\xae(\xbft\xc6\x1c?x+\x00\xff\\$\x93\xbe\x93\xb6$\xbfH[\xd2>e\x1f\x00\xff\xf7\x94\x93\xbe\x93\xb6$\xbfhK\xda>d\x1e\x00\xff\xcf\x1e\x95\xbe\x93\xb6$\xbf\x88;\xe2>]\x1d\x00\xff\xb8*\x98\xbe\x93\xb6$\xbf\xa8+\xea>V\x1b\x00\xff\x9

In [ ]:
# render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
# size = 64 # this is the size of the renders; higher values take longer to render.

# cameras = create_pan_cameras(size, device)
# for i, latent in enumerate(latents):
#     images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
#     display(gif_widget(images))